In [76]:
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np

In [77]:
def SetAlfa(x) : 
    global _alfa
    _alfa = x

In [78]:
def SetBeta(x) : 
    global _beta
    _beta = x

In [79]:
def Alfa() : return _alfa

In [80]:
def Beta() : return _beta

In [81]:
def FormatarFloat(x) : return round(x,4)

In [82]:
def CriarCamposTabela(x): return float(0), float(0),float(0), float(0),float(0) 

In [83]:
def RetornarValorSetupLB() : return FormatarFloat(df.loc[1]["yt"]), FormatarFloat(df.loc[2]["yt"] - df.loc[1]["yt"]) 

In [84]:
def AjustarValorSetupLB() : 
    df.set_value(0,"L", RetornarValorSetupLB()[0]) 
    df.set_value(0,"B", RetornarValorSetupLB()[1])

In [85]:
def CalcularL(yt,bAnt, lAnt ) : return FormatarFloat(Alfa()*yt+(1-Alfa())*(bAnt+lAnt))

In [86]:
def CalcularB(l, lAnt, bAnt ) : return FormatarFloat(Beta()*(l-lAnt)+(1-Beta())*bAnt)

In [87]:
def CalcularF(l, lAnt, bAnt ) : return FormatarFloat(Beta()*(l-lAnt)+(1-Beta())*bAnt)

In [88]:
def CalcularF(l, h, b) : return FormatarFloat(l+(h*b))

In [89]:
def CalcularE(yt, f) : return FormatarFloat(abs(yt-f))

In [90]:
def CalcularEPerc(e, yt) : return FormatarFloat((e/yt)*100)

In [91]:
def RetornarValorMAE() : return FormatarFloat(df["E"].mean())

In [92]:
def RetornarValorMAPE() : return FormatarFloat(df["E%"].mean())

In [93]:
def CalcularLinha(indice) :
    df.set_value(indice,"L", CalcularL(df.loc[indice]["yt"],df.loc[indice-1]["B"], df.loc[indice-1]["L"] ))
    df.set_value(indice,"B", CalcularB(df.loc[indice]["L"], df.loc[indice-1]["L"], df.loc[indice-1]["B"] ))
    df.set_value(indice,"F", CalcularF(df.loc[indice]["L"], 1, df.loc[indice]["B"] ))
    df.set_value(indice,"E", CalcularE(df.loc[indice]["yt"], df.loc[indice]["F"]))
    df.set_value(indice,"E%", CalcularEPerc(df.loc[indice]["E"], df.loc[indice]["yt"]))


In [94]:
def CalcularLinhas() :
        for indice in range(1, df.t.count()) :
                CalcularLinha(indice)

In [95]:
#Memória do modelo Holt Exponencial
def CriarLinhasPrevisoes(Qtde, dfOriginal) :
    lUltimo = df.loc[df.L.count()-1]["L"]
    bUltimo = df.loc[df.B.count()-1]["B"]
    countDF = df.t.count()-1
    for x in range(1, Qtde+1) :        
        data = [[countDF + x, 0, FormatarFloat(0), FormatarFloat(0), FormatarFloat(0), CalcularF(lUltimo, x, bUltimo), FormatarFloat(0), FormatarFloat(0)]]
        df2 = pd.DataFrame(data,columns=['t','Ano','yt','L','B','F','E','E%'])
        dfOriginal = dfOriginal.append(df2)
    return dfOriginal

In [96]:
def CalcularHoltExponencialSimples() :
    global MAE
    global MAPE
    MelhorAlfa = 0
    MelhorBeta = 0
    MenorPercentualErro = 9999999.99
    SetAlfa(0.1)
    SetBeta(0.9)
    #Incrementando Alfa e subtraindo Beta
    while (Alfa() <= 0.9) :
        CalcularLinhas()    
        MAE = RetornarValorMAE()    
        MAPE = RetornarValorMAPE()
        if (MenorPercentualErro > MAPE ):
            MelhorAlfa = Alfa()
            MelhorBeta = Beta()
            MenorPercentualErro = MAPE
        SetAlfa(Alfa() + 0.1)
        SetBeta(Beta() - 0.1)
        
    #Incrementando Beta e subtraindo Alfa
    SetAlfa(0.9)
    SetBeta(0.1)
    while (Beta() <= 0.9) :
        CalcularLinhas()    
        MAE = RetornarValorMAE()    
        MAPE = RetornarValorMAPE()
        if (MenorPercentualErro > MAPE ) :
            MelhorAlfa = Alfa()
            MelhorBeta = Beta()
            MenorPercentualErro = MAPE
        SetAlfa(Alfa() - 0.1)
        SetBeta(Beta() + 0.1)
        
    ##Apresenta o melhor modelo    
    SetAlfa(MelhorAlfa)
    SetBeta(MelhorBeta)
    CalcularLinhas()    
    MAE = RetornarValorMAE()    
    MAPE = RetornarValorMAPE()
    print ('Melhor Alfa: ' + str(FormatarFloat(MelhorAlfa) ));
    print ('Melhor Beta: ' + str(FormatarFloat(MelhorBeta) )); 
    ImprimirMAE_MAPE()
    return CriarLinhasPrevisoes(5, df)    
    

In [97]:
def ImprimirMAE_MAPE() :
    print ('Melhor MAE: ' + str(FormatarFloat(MAE) ));
    print ('Melhor MAPE: ' + str(FormatarFloat(MAPE) ));

In [98]:
#Testes das rotinas  - validação de valores
#CalcularL(9.75,9.75, -0.75 )
#CalcularB(9.6, 9.75, -0.75 )
#CalcularF(9.6, 1, -0.63)
#CalcularE(9.75, 8.97)
#CalcularEPerc(0.78, 9.75)
#CalcularLinha(1)

In [99]:
filepath = r'c:/projetos/dados/pasageiros.csv'

In [100]:
load = pd.read_csv(filepath,  sep=';').fillna(0)

In [101]:
df = load

In [102]:
#df

In [103]:
df["L"], df["B"], df["F"], df["E"], df["E%"]= zip(*df["yt"].map(CriarCamposTabela))

#Definição de valores NaN (NULO) para não impactar na média
df.set_value(0,"E", 'NaN')
df.set_value(0,"E%", 'NaN')

,t,Ano,yt,L,B,F,E,E%
0,0,1989,0.00,0.0,0.0,0.0,NaN,NaN
1,1,1990,17.55,0.0,0.0,0.0,0.0,0.0
2,2,1991,21.86,0.0,0.0,0.0,0.0,0.0
3,3,1992,23.89,0.0,0.0,0.0,0.0,0.0
4,4,1993,26.93,0.0,0.0,0.0,0.0,0.0
5,5,1994,26.89,0.0,0.0,0.0,0.0,0.0
6,6,1995,28.83,0.0,0.0,0.0,0.0,0.0
7,7,1996,30.08,0.0,0.0,0.0,0.0,0.0
8,8,1997,30.95,0.0,0.0,0.0,0.0,0.0
9,9,1998,30.19,0.0,0.0,0.0,0.0,0.0


In [104]:
#df

In [105]:
dfBeforeCalculos = df

In [106]:
AjustarValorSetupLB()

In [107]:
CalcularHoltExponencialSimples()

Melhor Alfa: 0.6
Melhor Beta: 0.4
Melhor MAE: 2.0898
Melhor MAPE: 8.0838


,t,Ano,yt,L,B,F,E,E%
0,0,1989,0.00,17.5500,4.3100,0.0000,NaN,NaN
1,1,1990,17.55,19.2740,3.2756,22.5496,4.9996,28.4877
2,2,1991,21.86,22.1358,3.1101,25.2459,3.3859,15.4890
3,3,1992,23.89,24.4324,2.7847,27.2171,3.3271,13.9267
4,4,1993,26.93,27.0448,2.7158,29.7606,2.8306,10.5110
5,5,1994,26.89,28.0382,2.0268,30.0650,3.1750,11.8074
6,6,1995,28.83,29.3240,1.7304,31.0544,2.2244,7.7156
7,7,1996,30.08,30.4698,1.4966,31.9664,1.8864,6.2713
8,8,1997,30.95,31.3566,1.2527,32.6093,1.6593,5.3612
9,9,1998,30.19,31.1577,0.6721,31.8298,1.6398,5.4316
